In [1]:
# import necesstray modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

In [2]:
df = pd.read_csv("../data/data_files.csv", encoding= 'unicode_escape') # read the data

In [3]:
import chardet # an ML mode, It uses machine learning to detect the encoding of a file

def process_urls_and_convert_to_features(url):
    response = requests.get(url)
    content = response.content
    
    encoding = chardet.detect(content)
    
    if encoding['encoding'] == "ISO-8859-1":
        return content.decode("iso-8859-1")
    return content.decode("utf-8") # convert the byte code generated to string


In [ ]:
df['Content'] = df['URL'].apply(lambda x: process_urls_and_convert_to_features(x))

In [ ]:
df

In [ ]:
X = df['Content']

In [ ]:
y = df['Name']

In [ ]:
from nltk.corpus import stopwords
from sklearn.base import BaseEstimator, TransformerMixin
import chardet # an ML mode, It uses machine learning to detect the encoding of a file

# Custom transformer for case-folding
class CaseFoldingTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return [x.lower() for x in X]

# Custom transformer for stop words removal
class StopWordsRemovalTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        stop_words_list = stopwords.words('english')
        return [' '.join([word for word in x.split() if word not in stop_words_list]) for x in X]
    
        
# Define the preprocessing steps
preprocessing_steps = [
    ('case_folding', CaseFoldingTransformer()),
    ('stop_words_removal', StopWordsRemovalTransformer())
]

In [ ]:
# Bag-of-words can be implemented using "CountVectorizer" in sklearn
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
#     ('preprocessing', Pipeline(steps=preprocessing_steps)),
    ('vectorizer', CountVectorizer(max_features=3000))
])

In [ ]:
X_train = X
y_train = y

In [ ]:
X_train_bow = pipeline.fit_transform(X_train)

In [ ]:
X_train[0]

In [ ]:
len( pipeline['vectorizer'].get_feature_names_out())

In [ ]:
pipeline['vectorizer'].get_feature_names_out()[100]

In [ ]:
import pickle

# Save the pipeline object
with open('preprocessing_pipeline.pkl', 'wb') as f:
    pickle.dump(pipeline, f)

In [ ]:
import os

print(os.getcwd())
print(os.path.dirname(os.getcwd()))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

clf.fit(X_train_bow, y_train)

In [ ]:
clf.predict(pipeline.transform(df['Content'][3]))

In [ ]:
df['Content'][3]

In [ ]:
y.shape